# Organize freezed data

12/12/2019 Fangming Xie
Check cell numbers


Aug 22th, 2019
Fangming Xie

- use qc-ed data as input
- select neurons only


In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_clst_utils
from scipy.io import mmread
from scipy import sparse
import time


In [2]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells'
# DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'

In [3]:
f = PATH_GENEBODY_ANNOTATION 
df_genes = pd.read_csv(f, sep="\t")
df_genes['gene_id'] = df_genes['gene_id'].apply(lambda x: x.split('.')[0])
df_genes = df_genes.set_index('gene_id')
df_genes_v2 = df_genes.reset_index().groupby('gene_name').first()
print(df_genes.shape)
df_genes.head()
print(df_genes_v2.shape)
df_genes_v2.head()

(53379, 6)
(53278, 6)


,gene_id,chr,start,end,strand,gene_type
gene_name,,,,,,
0610005C13Rik,ENSMUSG00000109644,chr7,45567794,45575327,-,antisense_RNA
0610006L08Rik,ENSMUSG00000108652,chr7,74818818,74853813,-,lincRNA
0610009B22Rik,ENSMUSG00000007777,chr11,51685386,51688874,-,protein_coding
0610009E02Rik,ENSMUSG00000086714,chr2,26445696,26459390,+,processed_transcript
0610009L18Rik,ENSMUSG00000043644,chr11,120348678,120351190,+,bidirectional_promoter_lncRNA


### 10x_nuclei_v3_macosko 
- 2 metadata .csv
- 1 data matrix in .rds format
- concatenate male and female, get autosomo genes only

In [49]:
dataset_name = '10x_nuclei_v3_macosko'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

print(f_meta)
# print(fout_meta)
print(f_data)
# print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/10x_nuclei_v3_macosko_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/10x_nuclei_v3_macosko_raw.npz


In [50]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(159738, 18)


,cluster_id,QC,cluster_label,cluster_color,class_label,subclass_label,size,gene.counts,umi.counts,Broad.QC.doublet,Broad.QC.Mito,Broad.passQC,MALE,Comb.QC,cl,nUMI,nGene,sex
10x_nuclei_v3_macosko_pBICCNsMMrMOpRMiM003d190318_AAACCCACACAAGCTT,90,1,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150,4859,2336,MALE
10x_nuclei_v3_macosko_pBICCNsMMrMOpRMiM003d190318_AAAGGATCAGACCGCT,90,19,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150,3172,1739,MALE
10x_nuclei_v3_macosko_pBICCNsMMrMOpRMiM003d190318_AAAGGATCATGGAACG,90,20,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150,6306,2760,MALE
10x_nuclei_v3_macosko_pBICCNsMMrMOpRMiM003d190318_AAAGGTAGTAATACCC,90,37,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150,3310,1806,MALE
10x_nuclei_v3_macosko_pBICCNsMMrMOpRMiM003d190318_AAAGTCCCAAGGTCAG,90,39,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150,2843,1543,MALE


In [6]:
# gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
# print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

In [56]:
print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))

113
76
37


In [7]:
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic        15924
Glutamatergic    85723
Non-Neuronal     58091
dtype: int64
(101647, 18)


In [12]:
# select cells

selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])


In [13]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

449.6917836666107


In [14]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### 10x_nuclei_v3

- gene_ids
- all cells passed QC (QC.csv) metadata

- all cells data
- output

In [57]:
dataset_name = '10x_nuclei_v3'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/10x_nuclei_v3_metadata.tsv


In [58]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(40166, 7)


,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
cell,,,,,,,
10x_nuclei_v3_AAACCCAAGCTCTTCC-1L8TX_181211_01_A02,59,1,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
10x_nuclei_v3_AAAGGGCTCGCGATCG-1L8TX_181211_01_A02,59,47,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
10x_nuclei_v3_AAAGTGACATCGCTGG-1L8TX_181211_01_A02,59,80,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
10x_nuclei_v3_AACAACCGTGCTCTCT-1L8TX_181211_01_A02,59,108,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
10x_nuclei_v3_AACAAGAAGCCTGCCA-1L8TX_181211_01_A02,59,112,L6b Shisa6,L6b,Glutamatergic,#2B9880,247


In [6]:
# gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
# print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(31053, 40166) (31053,) (40166,)


In [59]:
print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))

67
61
6


In [10]:
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic         6526
Glutamatergic    33180
Non-Neuronal       460
dtype: int64
(39706, 7)


In [8]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(31053, 39706) (31053,) (39706,)


In [9]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

134.8693606853485


In [10]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### 10x_cells_v3 

In [60]:
dataset_name = '10x_cells_v3'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

# print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

In [61]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(71183, 7)


,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
cell,,,,,,,
10x_cells_v3_AAACCCAAGCTTCATG-1L8TX_181211_01_G12,42,1,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
10x_cells_v3_AAACCCACACCAGCCA-1L8TX_181211_01_G12,42,3,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
10x_cells_v3_AAACGAACAACGATTC-1L8TX_181211_01_G12,42,8,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
10x_cells_v3_AAACGAATCTCGTGAA-1L8TX_181211_01_G12,42,11,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
10x_cells_v3_AAACGCTGTAGTCACT-1L8TX_181211_01_G12,42,14,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334


In [13]:
gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(31053, 71183) (31053,) (71183,)


In [62]:
print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))


78
68
10


In [13]:
# meta = meta[meta['class_label']!='Non-Neuronal']
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic         7908
Glutamatergic    61819
Non-Neuronal      1456
dtype: int64
(69727, 7)


In [15]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(31053, 69727) (31053,) (69727,)


In [16]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

532.1656138896942


In [17]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### 10x_cells_v2 

In [63]:
dataset_name = '10x_cells_v2'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

# print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

In [64]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(122641, 7)


,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
cell,,,,,,,
10x_cells_v2_AAACCTGAGGAGTCTG-1L8TX_171026_01_F03,9,1,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
10x_cells_v2_ACCTTTAGTACAGCAG-1L8TX_171026_01_F03,9,673,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
10x_cells_v2_ACGATACCACCCAGTG-1L8TX_171026_01_F03,9,724,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
10x_cells_v2_ACGGGTCAGTGGGCTA-1L8TX_171026_01_F03,9,885,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
10x_cells_v2_AGCAGCCCAGTTCATG-1L8TX_171026_01_F03,9,1205,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293


In [20]:
gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(31053, 122641) (31053,) (122641,)


In [67]:
print(meta['cluster_label'].unique())
print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))


['Sncg Slc17a8' 'Pvalb Calb1_1' 'L5 IT Rspo2_1' 'L5 IT Rspo1_1'
 'L6b Shisa6_2_1' 'L6 CT Cpa6' 'L5 NP Trh' 'Sncg Col14a1' 'L6 NP Trh_1'
 'Lamp5 Pdlim5' 'L6b Ror1' 'L5 IT S100b' 'L2/3 IT ' 'Vip Chat'
 'Vip C1ql1_1' 'L5 IT Tcap_1' 'Endo_3' 'L6 IT_2' 'Sst Myh8_2' 'Sst C1ql3_1'
 'L6 CT Nxph2 Pou3f2' 'Pvalb Reln' 'Lamp5 Slc35d3' 'Sst Calb2'
 'Pvalb Vipr2' 'L5 NP Slc17a8' 'Sst Pvalb Etv1_2' 'Sst Crhr2_1'
 'Pvalb Gpr149' 'L6 CT Gpr139' 'Sncg Npy2r' 'L6b Shisa6_1' 'Vip Lypd1'
 'Pvalb Il1rapl2' 'Sst C1ql3_3' 'Vip Serpinf1_2' 'Lamp5 Egln3_2' 'Vip Gpc3'
 'Sst Crhr2_2' 'Vip Htr1f_1' 'Vip Igfbp6' 'Sst Myh8_4' 'Lamp5 Egln3_1'
 'Pvalb Gabrg1' 'Vip Crispld2' 'Sst Hspe_3' 'Vip Mybpc1' 'Sst Myh8_3'
 'Astro Aqp4' 'Sst Hspe_2' 'Lamp5 Egln3_3' 'Lamp5 Lhx6' 'L6b Shisa6_2_2'
 'Vip C1ql1_2' 'L6b Kcnip1_2' 'L5 ET_1' 'L5 IT Tcap_2' 'L6 IT_1' 'L5 ET_3'
 'Endo_1' 'Oligo Opalin_1' 'L6 NP CT' 'Lamp5 Pax6' 'OPC Pdgfra'
 'L6 CT Nxph2 Kit' 'Sst Pvalb Etv1_3' 'Sst Hspe_1' 'Vip Htr1f_2'
 'Oligo Opalin_2' 'L5 ET_2' 'Sst 

In [16]:
# meta = meta[meta['class_label']!='Non-Neuronal']
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic        27588
Glutamatergic    93852
Non-Neuronal      1201
dtype: int64
(121440, 7)


In [22]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(31053, 121440) (31053,) (121440,)


In [23]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

510.46264123916626


In [24]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### Smart-cells

In [100]:
dataset_name = 'smarter_cells'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

# print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

In [101]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(6288, 128)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size,ar_id,exp_component_vendor_name,batch,...,Injection_type,NewBatch,Vendor,MultiPlex,percent_reads_aligned_to_rrna_rmsk,percent_reads_aligned_to_trna_rmsk,percent_reads_aligned_to_ncrna,percent_reads_aligned_to_introns,percent_reads_aligned_to_intergenic,percent_reads_aligned_unique
smarter_cells_LS-15395_S41_E1-50,19,1,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,543183476,LS-15395-41,R8S4-160812,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.162980,9.855384,3.264771,84.993663
smarter_cells_LS-15395_S42_E1-50,19,2,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,543183474,LS-15395-42,R8S4-160812,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.281620,6.763949,2.856108,87.252036
smarter_cells_LS-15501_S28_E1-50,19,94,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,544930823,LS-15501-28,R8S4-160817,...,0.0,HistoricalData,Covance,index96,NaN,NaN,17.066841,3.311346,2.437971,86.831969
smarter_cells_LS-15501_S81_E1-50,19,120,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,544930707,LS-15501-81,R8S4-160817,...,0.0,HistoricalData,Covance,index96,NaN,NaN,13.866006,11.769028,3.699682,85.252590
smarter_cells_LS-15524_S21_E1-50,19,267,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,549999237,LS-15524-21,R8S4-160906,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.912637,5.923318,2.385434,82.503705


In [87]:
gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(32324, 6288) (32324,) (6288,)


In [104]:
print(meta['cluster_label'].unique())
print(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_label'].unique())
print(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_label'].unique())

print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))

['Sst Myh8_1' 'L6 NP Trh' 'L5 NP Slc17a8' 'L5 IT S100b_2' 'Vip Chat_1'
 'Lamp5 Slc35d3' 'Sst Myh8_2' 'Sst Etv1' 'L5 IT Rspo1' 'L5 IT Rspo2'
 'L5 IT S100b_1' 'Vip Htr1f' 'Pvalb Reln' 'L5 IT Tcap_1' 'Vip Chat_2'
 'Vip Igfbp6' 'Lamp5 Egln3_1' 'L2/3 IT Macc1' 'L2/3 IT S100a6'
 'Vip Crispld2' 'Vip C1ql1' 'L2/3 IT Cdh13 Sla' 'L6 IT Sulf_1'
 'Lamp5 Pdlim5' 'Vip Chat_3' 'L6 IT Sulf_3' 'L5 IT Tcap_2' 'Astro Aqp4'
 'L6 CT Grp_1' 'L6 CT Nxph2_1' 'L6b Shisa6' 'L6b Ror1_1' 'L5 ET'
 'Sst Crhr2_1' 'Pvalb Vipr2' 'Sst C1ql3' 'Sncg' 'Sst Crhr2_2' 'Vip Gpc3'
 'Vip Serpinf1' 'Lamp5 Lhx6' 'Vip Lmo1' 'Pvalb Th' 'L6 CT Gpr139'
 'L6b Rprm' 'L6 IT Sulf_2' 'L5 IT Pld5' 'L6 CT Grp_2' 'Lamp5 Egln3_2'
 'Sst Chodl' 'L6 CT Nxph2_2' 'Sncg Slc17a8' 'L6 IT Car3' 'Pvalb Calb1'
 'L6b Ror1_2' 'Pvalb Gpr149' 'SMC Aoc3' 'L6b Kcnip1_1' 'L6b Kcnip1_2'
 'VLMC' 'Endo Slc38a5']
['Sst Myh8_1' 'L6 NP Trh' 'L5 NP Slc17a8' 'L5 IT S100b_2' 'Vip Chat_1'
 'Lamp5 Slc35d3' 'Sst Myh8_2' 'Sst Etv1' 'L5 IT Rspo1' 'L5 IT Rspo2'
 'L5 IT S100b

In [20]:
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label'].str.contains(r'^Glu|^GABA')]
# meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic        2092
Glutamatergic    4152
Non-Neuronal       44
dtype: int64
(6244, 128)


In [89]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(32324, 6244) (32324,) (6244,)


In [90]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

117.02609968185425


In [91]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### Smarter-nuclei

In [71]:
dataset_name = 'smarter_nuclei'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

# print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

In [72]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(6171, 145)


,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size,ar_id,exp_component_vendor_name,batch,...,pred.cl.2,pred.score.2,ref.cl.2,ref.cl.1.1,pred.cl.3,pred.score.3,cluster_label.2,category_label,Region.1,gene.counts
cell,,,,,,,,,,,,,,,,,,,,,
smarter_nuclei_SM-GE653_S113_E1-50,2,1,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869537,SM-GE653-S113,R8S4-171101,...,14,0.917036,10,31,14,0.917036,Vip Htr1f,GABAergic,Cortex,7792
smarter_nuclei_SM-GE653_S116_E1-50,2,4,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869531,SM-GE653-S116,R8S4-171101,...,3,0.712671,2,7,3,0.712671,Lamp5 Pdlim5,GABAergic,Cortex,8502
smarter_nuclei_SM-GE653_S117_E1-50,2,5,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869529,SM-GE653-S117,R8S4-171101,...,12,0.899050,9,28,12,0.899050,Vip Chat_3,GABAergic,Cortex,7021
smarter_nuclei_SM-GE653_S120_E1-50,2,8,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869523,SM-GE653-S120,R8S4-171101,...,11,0.892925,9,28,11,0.892925,Vip Chat_2,GABAergic,Cortex,7085
smarter_nuclei_SM-GE653_S122_E1-50,2,10,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869519,SM-GE653-S122,R8S4-171101,...,13,0.822251,9,28,13,0.822251,Vip Lmo1,GABAergic,Cortex,7516


In [55]:
gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(32324, 6171) (32324,) (6171,)


In [73]:
print(meta['cluster_label'].unique())
print(len(meta['cluster_id'].unique()))
print(len(meta.loc[meta['class_label']!='Non-Neuronal', 'cluster_id'].unique()))
print(len(meta.loc[meta['class_label']=='Non-Neuronal', 'cluster_id'].unique()))

['Lamp5 Egln3_2' 'Lamp5 Slc35d3' 'Lamp5 Pdlim5' 'Vip Cbln4' 'L2/3 IT'
 'Lamp5 Egln3_1' 'Sst Etv1_1' 'Pvalb Il1rapl2' 'Vip Rspo4' 'Vip Htr1f'
 'L5 IT Rspo1_1' 'Pvalb Vipr2' 'Vip Crispld2' 'Sncg' 'Sst Etv1_2'
 'L5 IT S100b_2' 'L6 CT' 'L5 NP Slc17a8' 'L6 IT Sulf_2' 'L6 IT Sulf_1'
 'L6 IT Fos' 'L6b Ror1' 'Pvalb Reln' 'L6b Kcnip1' 'L5 IT Rspo1_2'
 'L6b Shisa6' 'L5 ET_1' 'Lamp5 Lhx6' 'Sst Crhr2' 'Sst C1ql3' 'Sst Pvalb Th'
 'L6 NP Trh' 'Pvalb Sst Th' 'Sst Myh8' 'Pvalb Gpr149' 'Vip Serpinf1'
 'Oligo Opalin Serpinb1a' 'L5 IT S100b_1' 'L5 ET_3' 'Sst Chodl'
 'Astro Aqp4' 'Micro Siglech' 'L5 ET_2' 'Endo Slc38a5' 'OPC Pdgfra'
 'Oligo Opalin Cntn1']
46
40
6


In [23]:
# filter meta
print(meta.groupby('class_label').size())
meta = meta[meta['class_label'].str.contains(r'^Glu|^GABA')]
# meta = meta[meta['class_label']!='Non-Neuronal']
print(meta.shape)

class_label
GABAergic        1744
Glutamatergic    4167
Non-Neuronal      260
dtype: int64
(5911, 145)


In [57]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(32324, 5911) (32324,) (5911,)


In [58]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

51.310542821884155


In [12]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### mC

- QC'ed cells
- remove version on ensembl id

In [74]:
dataset_name = 'snmcseq_gene'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data_mc = os.path.join(SRC_DIR, dataset_name+'_mCH_raw.npz') 
f_data_c = os.path.join(SRC_DIR, dataset_name+'_CH_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

print(f_meta)
print(f_data_mc)
print(f_data_c)
print(f_data_gene)
print(f_data_cell)

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data_mc = os.path.join(DST_DIR, dataset_name+'_mCH_raw.npz') 
# fout_data_c = os.path.join(DST_DIR, dataset_name+'_CH_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 


# print(fout_meta)
# print(fout_data_mc)
# print(fout_data_c)
# print(fout_data_gene)
# print(fout_data_cell)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_mCH_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_CH_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.cell


In [90]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(9876, 31)


,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
cell,,,,,,,,,,,,,,,,,,,,,
snmcseq_gene_2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,2C,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
snmcseq_gene_2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,2C,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
snmcseq_gene_2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,2C,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
snmcseq_gene_2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,2C,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
snmcseq_gene_2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,2C,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [91]:
print(meta.columns.values)

['FullCellID' 'CCC_Rate' 'CG_Rate' 'CG_RateAdj' 'CH_Rate' 'CH_RateAdj'
 'FinalReads' 'InputReads' 'MappedReads' 'Region' 'index_name' 'uid'
 'BamFilteringRate' 'MappingRate' 'Pos96' 'Plate' 'Col96' 'Row96' 'Col384'
 'Row384' 'FACS_Date' 'Slice' 'PassFilter' 'pca_0' 'pca_1' 'umap_0'
 'umap_1' 'tsne_0' 'tsne_1' 'MajorCluster' 'SubCluster']


In [29]:
gc_mat = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
print(gc_mat.data['mc'].shape, gc_mat.data['c'].shape, gc_mat.gene.shape, gc_mat.cell.shape)

(55487, 9876) (55487, 9876) (55487,) (9876,)


In [92]:
meta[((meta['MajorCluster']!='Outlier') & (meta['SubCluster']=='Outlier'))].head()

,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
cell,,,,,,,,,,,,,,,,,,,,,
snmcseq_gene_2C_M_1090,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00544,0.76321,0.76191,0.02349,0.01815,2296305.0,4959968,3470043.0,2C,...,2,True,-16.348400,5.032563,7.670473,8.456449,-13.462711,20.760003,L4-IT-Rorb,Outlier
snmcseq_gene_2C_M_1135,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00540,0.77121,0.76997,0.02286,0.01755,2332977.0,5643100,3837928.0,2C,...,2,True,-12.510496,1.399961,7.740155,8.703683,-14.336394,20.211013,L4-IT-Rorb,Outlier
snmcseq_gene_2C_M_1137,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00557,0.77005,0.76876,0.02357,0.01810,2370160.0,5683314,3917031.0,2C,...,2,True,-16.943895,-2.267719,8.222825,9.749823,-15.304365,14.935951,L4-IT-Rorb,Outlier
snmcseq_gene_2C_M_1194,180508_CEMBA_mm_P56_P63_2C_CEMBA180410_2C_1_CE...,0.00556,0.75936,0.75801,0.02474,0.01929,577854.0,1330274,850069.0,2C,...,2,True,-6.064854,-13.665043,8.977352,1.497896,-11.189207,-13.304892,L23-IT-Cux2,Outlier
snmcseq_gene_2C_M_131,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00607,0.77063,0.76923,0.02344,0.01748,1671795.0,3883598,2682625.0,2C,...,2,True,-10.842959,5.305491,8.172520,6.193986,-1.238518,25.684641,L4-IT-Rorb,Outlier


In [94]:

print(meta['MajorCluster'].unique())
print(meta['SubCluster'].unique())
print(meta.loc[meta['MajorCluster'].str.contains(r'^L|^MGE|^CGE'), 'SubCluster'].unique())
print(meta.loc[meta['MajorCluster'].str.contains(r'^NonN'), 'SubCluster'].unique())

print(len(meta['SubCluster'].unique()))
print(len(meta.loc[meta['MajorCluster'].str.contains(r'^L|^MGE|^CGE'), 'SubCluster'].unique()))
print(len(meta.loc[meta['MajorCluster'].str.contains(r'^NonN'), 'SubCluster'].unique()))

['L4-IT-Rorb' 'L23-IT-Cux2' 'Outlier' 'CGE-VipNdnf' 'L6-IT-Sulf1'
 'L5-IT-Deptor' 'NonN' 'MGE-Sst' 'L6-CT-Foxp2' 'L6-NP-Tshz2' 'L5-PT-Bcl6'
 'MGE-Pvalb' 'L6b-Galnt10']
['L4-IT-Rorb_Rorb-Tenm2' 'L23-IT-Cux2' 'Outlier' 'CGE-VipNdnf_Lamp5-Unc5d'
 'L4-IT-Rorb_Rorb-Ryr3' 'CGE-VipNdnf_Pex5l' 'L4-IT-Rorb_Rorb-Cpne4'
 'L6-IT-Sulf1_Cables1' 'L5-IT-Deptor' 'NonN_Micro-Csf1r'
 'NonN_Astro-Slc1a2' 'NonN_OPC-Pdgfra' 'NonN_Endo-Nxn' 'MGE-Sst_Prep'
 'L6-CT-Foxp2_Spon1' 'L6-NP-Tshz2' 'L6-IT-Sulf1_Meis2'
 'L6-CT-Foxp2_Foxp2-Glra2' 'MGE-Sst_Daam2' 'MGE-Sst_Whrn' 'L5-PT-Bcl6'
 'CGE-VipNdnf_Vip-Unc5b' 'CGE-VipNdnf_Vip-Ano4' 'MGE-Pvalb_Man2a1'
 'MGE-Sst_Gfra2' 'MGE-Pvalb_Chrna7' 'CGE-VipNdnf_Vip-Sorcs1'
 'L6-CT-Foxp2_Foxp2-Kcnh5' 'MGE-Pvalb_Cnih3' 'NonN_VLMC-Slc6a13'
 'CGE-VipNdnf_Vip-Prr16' 'CGE-VipNdnf_Lamp5-Ndnf' 'NonN_ODC-Mog'
 'L6-IT-Sulf1_Pcdh15' 'L6-CT-Foxp2_Hcrtr2' 'MGE-Sst_Srrm4' 'MGE-Sst_Kcnip4'
 'NonN_SMC-Myo1b' 'L6-IT-Sulf1_Sulf1' 'L6-CT-Foxp2_Foxp2-Wscd1'
 'MGE-Sst_Chodl' 'MGE-Pvalb_Unc5b' 'L6

In [89]:
# meta[meta]
print(meta.groupby('MajorCluster').size())
print(meta.shape)
meta = meta.loc[meta['MajorCluster'].str.contains(r'^L|^MGE|^CGE')] # removed non-neuronal and outlier
print(meta.shape)
meta['MajorCluster'].unique()


MajorCluster
CGE-VipNdnf      625
L23-IT-Cux2     2057
L4-IT-Rorb      1582
L5-IT-Deptor     898
L5-PT-Bcl6       354
L6-CT-Foxp2     1622
L6-IT-Sulf1      881
L6-NP-Tshz2      275
L6b-Galnt10      141
MGE-Pvalb        460
MGE-Sst          471
NonN             428
Outlier           82
dtype: int64
(9876, 31)
(9366, 31)


array(['L4-IT-Rorb', 'L23-IT-Cux2', 'CGE-VipNdnf', 'L6-IT-Sulf1',
       'L5-IT-Deptor', 'MGE-Sst', 'L6-CT-Foxp2', 'L6-NP-Tshz2',
       'L5-PT-Bcl6', 'MGE-Pvalb', 'L6b-Galnt10'], dtype=object)

In [38]:
meta.groupby('MajorCluster').size()
print(meta.loc[meta['MajorCluster'].str.contains('^MGE|^CGE')].shape)
print(meta.loc[meta['MajorCluster'].str.contains('^L')].shape)

(1556, 31)
(7810, 31)


In [31]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

data_update = {'mc': gc_mat.data['mc'].tocsc()[:, selected_cells_id], 
               'c': gc_mat.data['c'].tocsc()[:, selected_cells_id], 
              }

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, data_update)
print(gc_mat_update.data['mc'].shape, 
      gc_mat_update.data['c'].shape, 
      gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(55487, 9366) (55487, 9366) (55487,) (9366,)


In [32]:
# save mc and c tables

snmcseq_utils.save_gc_matrix_methylation(gc_mat_update, 
                                         fout_data_gene, fout_data_cell, 
                                         fout_data_mc, fout_data_c)

In [33]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)

### ATAC 
- rename barcode and header
- read snap files

In [97]:
dataset_name = 'snatac_gene'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data = os.path.join(SRC_DIR, dataset_name+'_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

# fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
# fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
# fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
# fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

# print(f_meta)
# print(fout_meta)
# print(f_data)
# print(fout_data)

In [98]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(81196, 12)


,sample,barcode,TN,UM,PP,UQ,CM,cluster,tsne1,tsne2,umap-1,umap-2
cell,,,,,,,,,,,,
snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,138330,128263,128039,64868,0,L6.CT,7.605117,0.697672,-2.125204,-4.396653
snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,159133,146663,146348,69167,0,L23.b,-9.492925,-6.200938,8.959074,3.006819
snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152,17586,17545,8348,0,ASC,14.227358,-11.023748,-6.880754,-2.903271
snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876,13837,13801,6583,0,L6.CT,5.144768,4.790674,-0.955707,-4.635163
snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532,26303,26202,12437,0,L5.IT.b,5.218895,-8.674734,0.796851,4.447933


In [22]:
gc_mat = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data)
print(gc_mat.data.shape, gc_mat.gene.shape, gc_mat.cell.shape)

(53278, 81196) (53278,) (81196,)


In [99]:
print(len(meta['cluster'].unique()))
print(len(meta.loc[~meta['cluster'].str.contains(r'^OGC|^ASC|^MGC|^OPC|^Other|^Endo|^Smc'), 'cluster'].unique()))
print(len(meta.loc[meta['cluster'].str.contains(r'^OGC|^ASC|^MGC|^OPC|^Other|^Endo|^Smc'), 'cluster'].unique()))

33
26
7


In [45]:
print(meta['cluster'].value_counts())

OGC                8557
L6.CT              8008
L4                 7307
ASC                7215
L23.a              6085
L23.b              5758
L6.IT              4233
L5.IT.b            4214
MGC                3899
L5.IT.a            3668
L23.c              3590
OPC                3279
Pv_Tac1            1944
L5.PT              1755
Other              1571
Sst_Man1a          1355
NP                 1291
Endo               1245
Pv_Ntf3_Trim63     1105
Lamp5_Smad3         940
Smc                 586
Sst_Chrna2_Myh8     538
Vip_Chat            500
Pv_Vsig2            431
Vip_Hcls1           395
Vip_Gcnt4           292
Sncg                246
Sst_Stk33           246
Vip_Lipg            245
Lamp5_Arhgdib       237
Lamp5_Ndnf          204
Lamp5_Mettl21e      171
Chodl                86
Name: cluster, dtype: int64


In [48]:
print(meta.shape)
print(meta[meta['cluster'].str.contains(r'^OGC|^ASC|^MGC|^OPC|^Other|^Endo|^Smc')].shape)
print(meta[~meta['cluster'].str.contains(r'^OGC|^ASC|^MGC|^OPC|^Other|^Endo|^Smc')].shape)
print(meta[meta['cluster'].str.contains(r'^L[0-9]|NP')].shape)
print(meta[meta['cluster'].str.contains(r'^Pv|^Sst|^Lamp5|^Vip|^Sncg|^Chodl')].shape)

(81196, 12)
(26352, 12)
(54844, 12)
(45909, 12)
(8935, 12)


In [41]:
print(meta['cluster'].value_counts())
meta = meta[~meta['cluster'].str.contains(r'^OGC|^ASC|^MGC|^OPC|^Other|^Endo|^Smc')]
print(meta.shape)
print(meta['cluster'].unique())

OGC                8557
L6.CT              8008
L4                 7307
ASC                7215
L23.a              6085
L23.b              5758
L6.IT              4233
L5.IT.b            4214
MGC                3899
L5.IT.a            3668
L23.c              3590
OPC                3279
Pv_Tac1            1944
L5.PT              1755
Other              1571
Sst_Man1a          1355
NP                 1291
Endo               1245
Pv_Ntf3_Trim63     1105
Lamp5_Smad3         940
Smc                 586
Sst_Chrna2_Myh8     538
Vip_Chat            500
Pv_Vsig2            431
Vip_Hcls1           395
Vip_Gcnt4           292
Sncg                246
Sst_Stk33           246
Vip_Lipg            245
Lamp5_Arhgdib       237
Lamp5_Ndnf          204
Lamp5_Mettl21e      171
Chodl                86
Name: cluster, dtype: int64
(54844, 12)
['L6.CT' 'L23.b' 'L5.IT.b' 'L4' 'Pv_Ntf3_Trim63' 'NP' 'L5.IT.a' 'L23.a'
 'L5.PT' 'Sst_Man1a' 'L6.IT' 'Lamp5_Arhgdib' 'L23.c' 'Pv_Tac1'
 'Lamp5_Smad3' 'Vip_Hcls1' 'Sst_C

In [24]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells)
assert np.all(selected_cells_id != -1)

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, gc_mat.data.tocsc()[:, selected_cells_id])
print(gc_mat_update.data.shape, gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(53278, 54844) (53278,) (54844,)


In [25]:
# output
# save 
ti = time.time()
snmcseq_utils.save_gc_matrix(gc_mat_update, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

159.10273504257202


In [26]:
# assemble metadata (keep order, add male and female)
meta.index.name = 'cell'
meta.to_csv(fout_meta, sep="\t", header=True, index=True)